In [2]:
import numpy as np
import pickle
import cv2
import os
import keras
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import backend as K
import pickle
import joblib
from keras.models import model_from_json

In [3]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"


1 Physical GPUs, 1 Logical GPUs


In [4]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

In [5]:
ph1=r"C:\Users\aanis\Desktop\plantvillage1\aa\segmented\tomato"
li=0
x=[]
y=[]
for i in os.listdir(ph1):
    lab=[0,0,0,0,0,0,0,0,0,0]
    lab[li]=1
    ph2=ph1+f'\{i}'
    #print(ph2)
    li+=1
    for j in os.listdir(ph2):
        #print(j)
        ph3=ph2+f'\{j}'
        #print(x,np.array(cv2.imread(ph3)).reshape(256*256*3))
        #print(lab)
        x.append(np.array(cv2.imread(ph3)))
        y.append(lab)
        

In [6]:
x=(np.array(x))
print(np.array(y).shape)

(18159, 10)


In [7]:
print(x.shape)


(18159, 256, 256, 3)


In [8]:
width=256
height=256
depth=3
n_classes=10
EPOCHS = 50
INIT_LR = 1e-3
BS = 32

In [7]:
checkpoint_path="mldltest3train_ckpt/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_best_only=True)


NameError: name 'tf' is not defined

In [11]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))
model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 256, 256, 32)      896       
_________________________________________________________________
activation_14 (Activation)   (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 85, 85, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_15 (Activation)   (None, 85, 85, 64)       

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 42) 
del x
del y

NameError: name 'x' is not defined

In [12]:
model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [13]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1,
    callbacks=[cp_callback]
    )

C:\Users\aanis\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
453/453 [==============================] - 600s 1s/step - loss: 0.4379 - accuracy: 0.4625 - val_loss: 0.2698 - val_accuracy: 0.5862

Epoch 00001: val_loss improved from inf to 0.26978, saving model to mldltest3train_ckpt\cp.ckpt
Epoch 2/50
453/453 [==============================] - 573s 1s/step - loss: 0.2019 - accuracy: 0.6412 - val_loss: 0.3622 - val_accuracy: 0.4689

Epoch 00002: val_loss did not improve from 0.26978
Epoch 3/50
453/453 [==============================] - 574s 1s/step - loss: 0.1670 - accuracy: 0.6977 - val_loss: 0.1834 - val_accuracy: 0.6869

Epoch 00003: val_loss improved from 0.26978 to 0.18342, saving model to mldltest3train_ckpt\cp.ckpt
Epoch 4/50
402/453 [=========================>....] - ETA: 1:00 - loss: 0.1240 - accuracy: 0.7726

KeyboardInterrupt: 

In [ ]:
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
model.save("my_model_test2")

In [ ]:
train=model
acc = train.history['accuracy']
val_acc = train.history['val_accuracy']
loss = train.history['loss']
val_loss = train.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [17]:
reconstructed_model = keras.models.load_model("my_model")

In [18]:
scores = reconstructed_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'int\'>"})'})